In [1]:
import ssl
from pymongo import MongoClient
import json

In [2]:
#%% Pull Sensitive User URI String from local file (avoids having username and PW in code)
credDir = 'mongoURIHui.txt'       #contains user name and PW read data vice having sensitive info in code

In [3]:
#%% Now connect to the MongoDB reads one line from credentials file and uses
# it to connect to Mongo DB
with open(credDir,'r') as f:
    uri = f.readlines()

client = MongoClient(uri,ssl=True,ssl_cert_reqs=ssl.CERT_NONE)
db=client.IUReddit

In [4]:
# specify save dir
saveDir = 'D:/Depression_project/data/reddit' # local or network fileshare

In [37]:
#%% Global variables for saving, opening and naming
srList = ['mentalhealth']
strA = "AllPosts"
strB = "AllReddit"
strC = "Authors"
strD = "List"
strE = "Set"

In [39]:
#%% local functions
def save_json(fpath,var):
    """function to save a variable to json
    assumes variable data is compatible with python json methods"""
    with open(fpath,'w') as f:
        json.dump(var,f)  

def getMongoAuthSet(collection,srList):
    """
    Function to return an author set for a given collection object and list of subreddits
    assumes that the collection object is a a valid mongo collection object.
    Returns a list containing a list of authors and posts for that subreddit
    """    
    srauths = []
    srPosts = []
    count = 0
    postcount = 0
    for subreddit in srList:
        print ("Searching subreddit: ", subreddit)
        for doc in collection.find({"subreddit":subreddit},no_cursor_timeout=False):
            x = doc[u'author']
            postcount += 1
            srPosts.append(doc)
            if x not in srauths:
                srauths.append(x)
                count +=1
    print ("Total posts found: ",postcount)
    print ("Total authors found: ", count,"\n")
    return [srauths, srPosts]

In [14]:
#%% Now view collection names and get collection objects
names =  db.collection_names()  #get names of collections
yr = '2015'
names_yr = [x for x in names if yr in x]   #only pull collections from specified yr

In [31]:
names_yr

['RC_2015-11',
 'RC_2015-02',
 'RC_2015-10',
 'RC_2015-05',
 'RC_2015-08',
 'RC_2015-01',
 'RC_2015-04',
 'RC_2015-09',
 'RC_2015-12',
 'RC_2015-06',
 'RC_2015-07',
 'RC_2015-03']

In [ ]:
#%% Get list of authors from reddit posts
srAuths=[]                          # list of subreddit Authors
srPosts = []                        # container for subreddit posts
count = 0
postcount = 0
for c_name in names_yr:
    coll=db[c_name]             #returns a collection object
    print ("Current Collection: ",c_name)
    srauths,srposts = getMongoAuthSet(coll,srList)
    for auth in srauths:
        srAuths.append(auth)
    for item in srposts:
        srPosts.append(item)
srAuths = list(set(srAuths))

In [38]:
#%% Save the files
print ('\n retrieval complete, saving data to json')
save_json(saveDir+"/"+srList[0]+strC+strD+yr+".json",srAuths)
for doc in srPosts:
    doc.pop(u'_id',None)
save_json(saveDir+"/"+srList[0]+strC+strB+strA+yr+".json",srPosts)


 retrieval complete, saving data to json


# Collect random posts

In [5]:
# Get N random documents from the mycoll collection.
def getRandomPost(N_int, yr_string):
    posts = []
    #%% Now view collection names and get collection objects
    names =  db.collection_names()  #get names of collections
    names_yr = [x for x in names if yr_string in x]   #only pull collections from specified yr   
    for c_name in names_yr:
        coll=db[c_name]             #returns a collection object
        for doc in coll.aggregate([{ "$sample": { "size": N_int } }]):
            posts.append(doc)
    return posts

In [30]:
yr = '2017'
posts = getRandomPost(10000, yr)

In [31]:
len(posts)

120000

In [58]:
"""""
# text
posts = []
for doc in db['RC_2013-02'].aggregate([{ "$sample": { "size": 2 } }]):
    posts.append(doc)

In [32]:
def save_json(fpath,var):
    """function to save a variable to json
    assumes variable data is compatible with python json methods"""
    with open(fpath,'w') as f:
        json.dump(var,f)  

#%% Save the files
for doc in posts:
    doc.pop(u'_id',None)
save_json(saveDir+"/randomPosts"+yr+".json", posts)
print ('\n retrieval complete, saving data to json')


 retrieval complete, saving data to json
